In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
pwd

'/home/lucky/Documents/data_science_interview/Udemy/My_Projects/chicken_binary'

In [4]:
ls

artifacts/        dvc.yaml  params.yaml       setup.py     venv/
Chicken_project/  LICENSE   README.md         src/
config/           logs/     requirements.txt  template.py
data.zip          main.py   research/         templates/


# Entity

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class BaseModelConfig:
    base_model_dir: Path
    base_model_path: Path
    img_size: list
    channels: int
    loss: str
    learning_rate: float

# Configuration Manager

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH): 
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_base_model_config(self) ->BaseModelConfig:
        config = self.config.base_model
        params_config = self.params
        
        create_directories([config.base_model_dir])

        base_model_config = BaseModelConfig(
            base_model_dir=Path(config.base_model_dir),
            base_model_path=Path(config.base_model_path),
            img_size=params_config.img_size,
            channels=params_config.channels,
            loss=params_config.loss,
            learning_rate=params_config.learning_rate
        )

        return base_model_config

In [9]:
CONFIG_FILE_PATH

PosixPath('config/config.yaml')

In [10]:
PARAMS_FILE_PATH

PosixPath('params.yaml')

In [11]:
read_yaml(PARAMS_FILE_PATH).img_size

[2023-12-10 10:47:06,306: INFO: common: yaml file: params.yaml loaded successfully]


BoxList([224, 224])

In [12]:
temp = ConfigurationManager().get_base_model_config()

[2023-12-10 10:47:06,317: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-10 10:47:06,321: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-10 10:47:06,323: INFO: common: created directory at: artifacts]
[2023-12-10 10:47:06,324: INFO: common: created directory at: artifacts/base_model]


In [13]:
temp

BaseModelConfig(base_model_dir=PosixPath('artifacts/base_model'), base_model_path=PosixPath('artifacts/base_model/base_model.h5'), img_size=BoxList([224, 224]), channels=3, loss='binary_crossentropy', learning_rate=0.001)

# Prepare Model

In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, Adamax

2023-12-10 10:47:06.549694: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-10 10:47:06.582438: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-10 10:47:06.582469: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-10 10:47:06.583297: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-10 10:47:06.588289: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-10 10:47:06.588836: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [15]:
class PrepareBaseModel:
    def __init__(self, config: BaseModelConfig):
        self.model = None
        self.config = config


    def prepare_model(self):
        input_shape = (self.config.img_size[0], self.config.img_size[1], self.config.channels)
        learning_rate = self.config.learning_rate
        loss = self.config.loss
        
        model = Sequential([
                Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape),
                BatchNormalization(axis= -1, momentum= 0.99, epsilon= 0.001),
                MaxPooling2D(pool_size=2, strides=2),
                Conv2D(filters=32, kernel_size=3, activation='relu'),
                MaxPooling2D(pool_size=2, strides=2),
                Flatten(),
                Dense(units=128, activation='relu'),
                Dense(units=1, activation='sigmoid')
            ])
        model.compile(Adamax(learning_rate=learning_rate), loss=loss, metrics= ['accuracy'])
        model.summary()
        self.model = model
        self.save_model(path=self.config.base_model_path, model=self.model)
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [16]:
try:
    config = ConfigurationManager()
    base_model_config = config.get_base_model_config()
    base_model_obj = PrepareBaseModel(config=base_model_config)
    base_model_obj.prepare_model()
except Exception as e:
    raise e

[2023-12-10 10:47:08,178: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-10 10:47:08,181: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-10 10:47:08,182: INFO: common: created directory at: artifacts]
[2023-12-10 10:47:08,183: INFO: common: created directory at: artifacts/base_model]
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 batch_normalization (Batch  (None, 222, 222, 32)      128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                      

/home/lucky/Documents/data_science_interview/Udemy/My_Projects/chicken_binary/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
